In [1]:
import os
import re
import time
import pandas as pd
import numpy as np

start = time.perf_counter()

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:
train_X_df = pd.DataFrame()
test_X_df = pd.DataFrame()

In [3]:
# strで書かれてしまったリストをndarrayに変換する
def str_to_nd(strs):
    strs = re.sub('[{}"]','',strs)
    str_list = strs.split(',')
    str_nd = np.array(str_list)
    return str_nd

# リストをOne-Hotに変換する
from sklearn.preprocessing import MultiLabelBinarizer
def list_to_onehot(train,test):
    mlb = MultiLabelBinarizer()
    train_onehot = mlb.fit_transform(train.values)
    test_onehot = mlb.transform(test.values)
    train_onehot_df = pd.DataFrame(train_onehot, columns = mlb.classes_)
    columns_list = train_onehot_df.mean()[train_onehot_df.mean()>0.00].index.tolist()
    train_onehot_df = train_onehot_df[columns_list]
    test_onehot_df = pd.DataFrame(test_onehot, columns = mlb.classes_)
    test_onehot_df = test_onehot_df[columns_list]
    return train_onehot_df.astype('int').astype('category'),test_onehot_df.astype('int').astype('category')

# アメニティデータ
train['amenities'] = train['amenities'].map(str_to_nd)
test['amenities'] = test['amenities'].map(str_to_nd)
train_X_df = pd.concat([train_X_df,list_to_onehot(train['amenities'],test['amenities'])[0]], axis=1)
test_X_df = pd.concat([test_X_df,list_to_onehot(train['amenities'],test['amenities'])[1]], axis=1)
# アメニティの数
train_X_df = pd.concat([train_X_df,train['amenities'].map(lambda x:len(x)).rename('am_num')], axis=1)
test_X_df = pd.concat([test_X_df,test['amenities'].map(lambda x:len(x)).rename('am_num')], axis=1)

C:\Users\sitc_trainee_03\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:994: UserWarning: unknown class(es) ['Wide clearance to shower and toilet'] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'
C:\Users\sitc_trainee_03\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:994: UserWarning: unknown class(es) ['Wide clearance to shower and toilet'] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'


In [4]:
# カテゴリー系のデータをラベルにする
import itertools
import category_encoders as ce
def df_to_label(train,test):
    train,test = train.fillna('nan'),test.fillna('nan')
    oe = ce.OrdinalEncoder(handle_unknown='return nan')
    train_oe = oe.fit_transform(train).fillna(999).astype(int).astype('category') # 欠損値はありえない数値で埋める
    test_oe = oe.transform(test).fillna(999).astype(int).astype('category')
    return train_oe,test_oe

# カテゴリー系データ
cat_list = ['bed_type','cancellation_policy','city','cleaning_fee','host_has_profile_pic','host_identity_verified','instant_bookable','property_type','room_type','neighbourhood']
train_X_df = pd.concat([train_X_df,df_to_label(train[cat_list],test[cat_list])[0]], axis=1)
test_X_df = pd.concat([test_X_df,df_to_label(train[cat_list],test[cat_list])[1]], axis=1)

In [5]:
# 日付関連のデータを整形
import datetime
def str_to_date(df):
    # 年と月だけを抜き出す、nanはそのまま
    df_year = df.applymap(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').year if x==x else x)
    df_month = df.applymap(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').month if x==x else x)

    # 月で換算する(わかりやすくするために年は2000までは切り捨て)
    df_date = (df_year - 2000) * 12 + df_month

    return df_date

# 日付関連のデータ
date_list = ['host_since','first_review','last_review']
train_X_df = pd.concat([train_X_df,str_to_date(train[date_list])], axis=1)
test_X_df = pd.concat([test_X_df,str_to_date(test[date_list])], axis=1)

In [6]:
# 月データ(季節)
str_to_month = lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').month if x==x else x
train_X_df = pd.concat([train_X_df,train['host_since'].map(str_to_month).rename('host_since_month')], axis=1)
test_X_df = pd.concat([test_X_df,test['host_since'].map(str_to_month).rename('host_since_month')], axis=1)

train_X_df = pd.concat([train_X_df,train['first_review'].map(str_to_month).rename('first_review_month')], axis=1)
test_X_df = pd.concat([test_X_df,test['first_review'].map(str_to_month).rename('first_review_month')], axis=1)

train_X_df = pd.concat([train_X_df,train['last_review'].map(str_to_month).rename('last_review_month')], axis=1)
test_X_df = pd.concat([test_X_df,test['last_review'].map(str_to_month).rename('last_review_month')], axis=1)

In [7]:
# 反応があるかないかをOne-Hotデータで表現する
def res_to_onehot(df):
    df = df.applymap(lambda x: 1 if x==x else 0)
    df = df.set_axis([x+'_hot' for x in df.columns], axis=1)
    return df.astype('category')

#res_list = ['first_review','host_since','last_review','thumbnail_url']
res_list = ['host_since']
train_X_df = pd.concat([train_X_df,res_to_onehot(train[res_list])], axis=1)
test_X_df = pd.concat([test_X_df,res_to_onehot(test[res_list])], axis=1)

In [8]:
# str型のパーセンテージで書かれたデータを0~1のfloat型にする
def strper_to_float(strper):
    if strper==strper:
        strnum = re.sub('%','',strper)
        num_float = float(strnum) / 100.
    else:
        num_float = np.nan
    return num_float

# str型のパーセンテージ
train['host_response_rate'] = train['host_response_rate'].map(strper_to_float)
test['host_response_rate'] = test['host_response_rate'].map(strper_to_float)

# float型データ
num_list = ['accommodates','bathrooms','bedrooms','beds','latitude','longitude','number_of_reviews','review_scores_rating','host_response_rate']
train_X_df = pd.concat([train_X_df,train[num_list]], axis=1)
test_X_df = pd.concat([test_X_df,test[num_list]], axis=1)

In [9]:
# 緯度経度から地理情報を入手
from uszipcode import SearchEngine
search = SearchEngine()
def latlon_to_geo(lat, lon):
    d = search.by_coordinates(lat, lon, radius=20)[0].to_dict()
    return pd.DataFrame([d])

def data_to_geo(df):
    geo = pd.concat([latlon_to_geo(x,y) for x,y in zip(df['latitude'],df['longitude'])])
    return geo

def geo_cat_num(geo):
    geo['zipcode'] = geo['zipcode'].apply(lambda x:str(x)[:5])
    cat_list = ['zipcode', 
                'major_city']

    num_list = ['population', 
                'population_density', 
                'land_area_in_sqmi', 
                'water_area_in_sqmi', 
                'housing_units', 
                'occupied_housing_units', 
                'median_home_value', 
                'median_household_income']
    
    return geo[cat_list], geo[num_list]

# DataFrameの作成が非常に遅いため、DataFrameを作成したらcsvとして保存し次に使うときはcsvから読み込む

if os.path.isfile('train_geo.csv'):
    train_geo = pd.read_csv('train_geo.csv', index_col=0)
else:
    train_geo = data_to_geo(train).reset_index()
    train_geo.to_csv('train_geo.csv', header=True, index=True)
train_geo_cat, train_geo_num = geo_cat_num(train_geo)

if os.path.isfile('test_geo.csv'):
    test_geo = pd.read_csv('test_geo.csv', index_col=0)
else:
    test_geo = data_to_geo(test).reset_index()
    test_geo.to_csv('test_geo.csv', header=True, index=True)
test_geo_cat, test_geo_num = geo_cat_num(test_geo)

train_X_df = pd.concat([train_X_df,df_to_label(train_geo_cat,test_geo_cat)[0]], axis=1)
test_X_df = pd.concat([test_X_df,df_to_label(train_geo_cat,test_geo_cat)[1]], axis=1)
train_X_df = pd.concat([train_X_df,train_geo_num], axis=1)
test_X_df = pd.concat([test_X_df,test_geo_num], axis=1)

In [10]:
# descriptionの単語数を調べる
from collections import Counter
wordnum = lambda x:sum(Counter(x.split()).values())
train_X_df = pd.concat([train_X_df,train['description'].map(wordnum).rename('word_num')], axis=1)
test_X_df = pd.concat([test_X_df,test['description'].map(wordnum).rename('word_num')], axis=1)

In [11]:
# 部屋数
train_X_df['rooms'] = train_X_df['bedrooms'] + train_X_df['bathrooms']
test_X_df['rooms'] = test_X_df['bedrooms'] + test_X_df['bathrooms']

In [12]:
# 価格が高い地点との距離を測る
top_loc = np.array([[-118.813009,34.028313],[-118.448221,34.132164]])

def u_dis(x1,y1,x2,y2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

def min_dis(df):
    tmp = np.empty(2)
    for i,x,y in zip(range(1),top_loc[:,0],top_loc[:,1]):
        tmp[i] = u_dis(df['longitude'],df['latitude'],x,y)
    return tmp

train_X_df = pd.concat([train_X_df,train_X_df.apply(min_dis, axis=1).apply(pd.Series)], axis=1)
test_X_df = pd.concat([test_X_df,test_X_df.apply(min_dis, axis=1).apply(pd.Series)], axis=1)

train_X_df = train_X_df.rename(columns=dict([(x,'dis_'+str(x)) for x in range(2)]))
test_X_df = test_X_df.rename(columns=dict([(x,'dis_'+str(x)) for x in range(2)]))

In [13]:
# 占有住宅率
train_X_df['occupied_housing_rate'] = train_X_df['occupied_housing_units'] / train_X_df['housing_units']
test_X_df['occupied_housing_rate'] = test_X_df['occupied_housing_units'] / test_X_df['housing_units']

In [14]:
# 学習用データの作成
from nltk import sent_tokenize, word_tokenize
remnum = lambda x:re.sub(r'[0-9]+', " ", x)
remstr = lambda x:re.sub(r'[\．_－―─\-‐|\“■×+α÷⇒—●□(=)*&^%$#@!~`){}…\[\]\"\'\”\’:;<>?・,\./→←○\n\u3000]+', " ", x)
shape_sent = lambda x:word_tokenize(x.lower())

train_desc = train['description'].map(remnum).map(remstr).map(shape_sent)
test_desc = test['description'].map(remnum).map(remstr).map(shape_sent)

train_name = train['name'].map(remnum).map(remstr).map(shape_sent)
test_name = test['name'].map(remnum).map(remstr).map(shape_sent)

In [15]:
# description用モデルの作成
vector_size=50
from nltk import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
if os.path.isfile('description.model'):
    model_desc = Word2Vec.load('description.model')
else:
    # word2vecのモデル作成
    sentence = train_desc.values.tolist()
    model_desc = Word2Vec(sentence,vector_size=vector_size,window=15,min_count=5)
    model_desc.train(sentence,total_examples=len(sentence),epochs=100)
    model_desc.save('description.model')

C:\Users\sitc_trainee_03\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [16]:
# name用モデルの作成
vector_size_name=50
if os.path.isfile('name.model'):
    model_name = Word2Vec.load('name.model')
else:
    # word2vecのモデル作成
    sentence = train_name.values.tolist()
    model_name = Word2Vec(sentence,vector_size=vector_size,window=10,min_count=5)
    model_name.train(sentence,total_examples=len(sentence),epochs=50)
    model_name.save('name.model')

In [17]:
# descriptionの文章ベクトルの作成
def wordvec2docvec(sentence):
    # 文章ベクトルの初期値（0ベクトルを初期値とする）
    docvecs = np.zeros(vector_size, dtype="float32")

    # 文章の中に存在する単語の数
    denomenator = len(sentence)
    # 文章内の各単語ベクトルを足し合わせる
    for word in sentence:
        try:
            temp = model_desc.wv[word]
        except:
            denomenator -= 1
            continue
        docvecs += temp

    # 文章に現れる単語のうち、モデルに存在した単語の数で割る
    if denomenator > 0:
        docvecs =  docvecs / denomenator

    return docvecs

train_X_df = pd.concat([train_X_df,train_desc.map(wordvec2docvec).apply(pd.Series)], axis=1)
test_X_df = pd.concat([test_X_df,test_desc.map(wordvec2docvec).apply(pd.Series)], axis=1)
train_X_df = train_X_df.rename(columns=dict([(x,'desc_'+str(x)) for x in range(vector_size)]))
test_X_df = test_X_df.rename(columns=dict([(x,'desc_'+str(x)) for x in range(vector_size)]))

In [18]:
# nameの文章ベクトルの作成
def wordvec2docvec(sentence):
    # 文章ベクトルの初期値（0ベクトルを初期値とする）
    docvecs = np.zeros(vector_size_name, dtype="float32")

    # 文章の中に存在する単語の数
    denomenator = len(sentence)
    # 文章内の各単語ベクトルを足し合わせる
    for word in sentence:
        try:
            temp = model_name.wv[word]
        except:
            denomenator -= 1
            continue
        docvecs += temp

    # 文章に現れる単語のうち、モデルに存在した単語の数で割る
    if denomenator > 0:
        docvecs =  docvecs / denomenator

    return docvecs

train_X_df = pd.concat([train_X_df,train_name.map(wordvec2docvec).apply(pd.Series)], axis=1)
test_X_df = pd.concat([test_X_df,test_name.map(wordvec2docvec).apply(pd.Series)], axis=1)
train_X_df = train_X_df.rename(columns=dict([(x,'name_'+str(x)) for x in range(vector_size)]))
test_X_df = test_X_df.rename(columns=dict([(x,'name_'+str(x)) for x in range(vector_size)]))

KeyboardInterrupt: 

In [20]:
pd.set_option('display.max_columns', 800)
display(train_X_df.head(5))
print(train_X_df.shape)

,24-hour check-in,Air conditioning,Bathtub,Bed linens,Breakfast,Buzzer/wireless intercom,Cable TV,Carbon monoxide detector,Cat(s),Children’s books and toys,Coffee maker,Cooking basics,Dishes and silverware,Dishwasher,Dog(s),Doorman,Dryer,Elevator,Elevator in building,Essentials,Extra pillows and blankets,Family/kid friendly,Fire extinguisher,First aid kit,Free parking on premises,Garden or backyard,Gym,Hair dryer,Hangers,Heating,Host greets you,Hot tub,Hot water,Indoor fireplace,Internet,Iron,Keypad,Kitchen,Laptop friendly workspace,Lock on bedroom door,Lockbox,Long term stays allowed,Luggage dropoff allowed,Microwave,Oven,Pack ’n Play/travel crib,Patio or balcony,Pets allowed,Pets live on this property,Pool,Private entrance,Private living room,Refrigerator,Room-darkening shades,Safety card,Self Check-In,Shampoo,Smartlock,Smoke detector,Smoking allowed,Step-free access,Stove,Suitable for events,TV,Washer,Well-lit path to entrance,Wheelchair accessible,Wireless Internet,translation missing: en.hosting_amenity_49,translation missing: en.hosting_amenity_50,am_num,bed_type,cancellation_policy,city,cleaning_fee,host_has_profile_pic,host_identity_verified,instant_bookable,property_type,room_type,neighbourhood,host_since,first_review,last_review,host_since_month,first_review_month,last_review_month,host_since_hot,accommodates,bathrooms,bedrooms,beds,latitude,longitude,number_of_reviews,review_scores_rating,host_response_rate,zipcode,major_city,population,population_density,land_area_in_sqmi,water_area_in_sqmi,housing_units,occupied_housing_units,median_home_value,median_household_income,word_num,rooms,dis_0,dis_1,occupied_housing_rate,desc_0,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,desc_8,desc_9,desc_10,desc_11,desc_12,desc_13,desc_14,desc_15,desc_16,desc_17,desc_18,desc_19,desc_20,desc_21,desc_22,desc_23,desc_24,desc_25,desc_26,desc_27,desc_28,desc_29,desc_30,desc_31,desc_32,desc_33,desc_34,desc_35,desc_36,desc_37,desc_38,desc_39,desc_40,desc_41,desc_42,desc_43,desc_44,desc_45,desc_46,desc_47,desc_48,desc_49
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,7,1,1,1,1,1,1,1,1,1,1,199.0,199.0,199.0,7.0,7.0,7.0,1,6,2.0,1.0,4.0,33.788931,-118.154761,1,60.0,NaN,1,1,40311,19061.0,2.11,0.00,15792,14556,340600.0,42672.0,76,3.0,0.700424,-6.522993e-166,0.921733,-1.900919,0.231837,-0.361965,-0.156130,-0.888952,-0.208904,-1.498402,-1.058776,-1.216541,0.651656,-0.626779,0.310452,-0.587204,0.303202,0.293937,0.174701,0.426225,-0.731825,-0.458716,-0.847465,0.128600,-0.935357,-1.557620,0.343270,-0.900722,2.083479,-1.143699,0.372548,0.619828,1.278649,0.188055,0.684179,-0.713492,1.411318,-0.574032,0.528712,-0.978824,-0.493474,-1.327512,-1.043061,-0.318801,-0.975571,0.215525,0.838071,0.070018,-0.534130,-0.801854,0.992161,-0.442845,1.611833
1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,1,1,22,1,2,2,1,1,2,1,2,1,2,192.0,201.0,207.0,12.0,9.0,3.0,1,2,1.0,1.0,1.0,38.934810,-76.978190,9,100.0,1.0,2,2,16894,5602.0,3.02,0.01,7652,7005,355500.0,52317.0,70,2.0,42.121560,1.061916e+109,0.915447,0.044158,0.113658,-0.033609,-0.364714,0.567105,0.556047,-0.630063,1.355203,0.906076,1.044096,-0.557932,-0.351208,-0.424376,-0.229480,-0.556773,-0.920768,-0.316864,-0.651414,-1.711163,-0.016381,-0.703694,0.295607,-0.946121,0.525093,-0.734590,1.056076,-0.226515,0.039734,-0.553441,1.237357,0.458374,0.601546,0.402689,0.694649,-0.346330,-1.309196,-0.088227,1.793174,0.442358,-0.552676,0.271393,0.220107,-0.050281,-0.068751,-0.194533,-0.259209,0.293063,-0.529087,-0.901656,-0.065665
2,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,0,0,0,0,1,1,0,0,1,1,1,29,1,2,3,1,1,1,2,1,1,3,197.0,198.0,212.0,5.0,6.0,8.0,1,2,2.0,1.0,1.0,40.695118,-73.926240,27,83.0,1.0,3,3,78895,57033.0,1.38,0.00,31784,28391,543800.0,39178.0,79,3.0,45.379162,

In [22]:
from copy import copy
import catboost
from catboost import Pool
import sklearn.preprocessing as sp
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
def CAT(train_X_df,train_Y_df,test_X_df,test_size=0):
    # 代入されないようシャドーコピー
    train_X,train_Y,test_X = copy(train_X_df),copy(train_Y_df),copy(test_X_df)
    # データ分割(test_sizeを指定しない場合は分割しない)
    if test_size:
        train_X, train_X_split, train_Y, train_Y_split = train_test_split(train_X, train_Y, test_size=test_size)
    # データを変換
    cat_features = train_X.dtypes[train_X.dtypes == 'category'].index
    c_train = Pool(train_X, label=train_Y,cat_features=cat_features)
    if test_size: c_train_split = Pool(train_X_split, label=train_Y_split,cat_features=cat_features)
    c_test = Pool(test_X,cat_features=cat_features)
    # モデル定義
    model = catboost.CatBoostRegressor(iterations=100000, loss_function='RMSE')
    # 学習
    if test_size:
        model.fit(c_train, eval_set=c_train_split,early_stopping_rounds=1000, use_best_model=True,verbose=True)
    else:
        model.fit(c_train, early_stopping_rounds=1000, use_best_model=True,verbose=True)

    # モデルの評価(データ分割をしている場合としてしない場合で分岐)
    
    if test_size:
        pred_train = model.predict(c_train)
        pred_test = model.predict(c_train_split)
        print(mean_squared_error(train_Y, pred_train))
        print(mean_squared_error(train_Y_split, pred_test))
    else:
        pred_train = model.predict(c_train)
        print(mean_squared_error(train_Y, pred_train))

    # モデルの予測
    predictions = model.predict(c_test)

    return predictions, model.get_feature_importance(c_train), train_X.columns

60 (658)	total: 3m 18s	remaining: 37.2s
842:	learn: 77.3369845	test: 99.2745396	best: 99.1285460 (658)	total: 3m 18s	remaining: 37s
843:	learn: 77.3332779	test: 99.2755531	best: 99.1285460 (658)	total: 3m 18s	remaining: 36.7s
844:	learn: 77.3237808	test: 99.2777809	best: 99.1285460 (658)	total: 3m 19s	remaining: 36.5s
845:	learn: 77.3101861	test: 99.2791128	best: 99.1285460 (658)	total: 3m 19s	remaining: 36.3s
846:	learn: 77.2951691	test: 99.2759221	best: 99.1285460 (658)	total: 3m 19s	remaining: 36s
847:	learn: 77.2536982	test: 99.2847200	best: 99.1285460 (658)	total: 3m 19s	remaining: 35.8s
848:	learn: 77.2336665	test: 99.2777741	best: 99.1285460 (658)	total: 3m 19s	remaining: 35.5s
849:	learn: 77.2091788	test: 99.2646954	best: 99.1285460 (658)	total: 3m 20s	remaining: 35.3s
850:	learn: 77.1977737	test: 99.2768809	best: 99.1285460 (658)	total: 3m 20s	remaining: 35.1s
851:	learn: 77.1831332	test: 99.2805788	best: 99.1285460 (658)	total: 3m 20s	remaining: 34.9s
852:	learn: 77.1539492	t

In [ ]:
model_num = 100 # モデルの数
timeout = 100000 # 何秒経ったら切り上げるか

dir_path = '/submit/subs'

os.makedirs(dir_path, exist_ok=True)

for i in range(model_num):
    predictions, feature_importances, feature_names = CAT(train_X_df,train['y'],test_X_df,test_size=0.1)
    results = predictions.flatten()
    pd.DataFrame(results).to_csv(os.path.join(dir_path, str(i+1) + 'sub.csv'), header=False, index=True)
    if (time.perf_counter() - start) > timeout: break

In [ ]:
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

In [48]:
from glob import glob

csvs = glob(dir_path + '*')
csv_list = [pd.read_csv(csv, header=None) for csv in csvs]
sum_csv = sum(csv_list) / len(csv_list)

pd.DataFrame(sum_csv).to_csv('submit.csv', header=False, index=True)